In [ ]:
import nltk
import pandas as pd
import numpy as np
import re
import emoji

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn import metrics
import datasets
import torch
from transformers import AutoTokenizer, AutoModel
nltk.download('stopwords')
nltk.download('wordnet')

In [98]:
train = pd.read_csv('./dataset/train.csv')
test = pd.read_csv('./dataset/test.csv')

dataset = datasets.load_dataset('csv', data_files={'train':'./dataset/train.csv','test': './dataset/test.csv'})
dataset = dataset.remove_columns(['Unnamed: 0', 'title', 'author', 'num_comments', 'url', 'selftext_clean', 'title_clean', 'author_clean', 'selftext_length', 'title_length', 'megatext_clean'])
dataset = dataset.rename_column('is_suicide', 'label')
dataset = dataset.rename_column('selftext', 'text')
# print(dataset['train']['text'])

Using custom data configuration default-0357399e0848af31
Reusing dataset csv (C:\Users\anca.ilicea\.cache\huggingface\datasets\csv\default-0357399e0848af31\0.0.0\6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)



100%|██████████| 2/2 [00:00<00:00, 286.38it/s]


In [99]:
train

,Unnamed: 0,title,selftext,author,num_comments,is_suicide,url,selftext_clean,title_clean,author_clean,selftext_length,title_length,megatext_clean
0,339,Need help,Hi I don't really know how to phrase this situ...,F4GG0T_,6,0,https://www.reddit.com/r/depression/comments/f...,hi really know phrase situation try life reall...,need help,f 4 gg 0,2502,9,f 4 gg 0 hi really know phrase situation try l...
1,1386,feeling so overwhelmed and hopeless,i have been so depressed these past couple wee...,thiccbarbie420,2,1,https://www.reddit.com/r/SuicideWatch/comments...,depressed past couple week ever since got back...,feeling overwhelmed hopeless,th icc barbie 420,843,35,th icc barbie 420 depressed past couple week e...
2,544,"Nothing matters anymore, getting worse",Hi..I don't know where else to go. I am devast...,mjdg25,3,0,https://www.reddit.com/r/depression/comments/f...,hi know else go devastated right feeling like ...,nothing matter anymore getting worse,mj g 25,1909,38,mj g 25 hi know else go devastated right feeli...
3,937,Who’s tired of hearing bullshit,"The shit like “it will get better, everyone is...",fuckxboxandgames,4,1,https://www.reddit.com/r/SuicideWatch/comments...,shit like get better everyone purpose need fin...,tired hearing bullshit,fuck xbox game,403,31,fuck xbox game shit like get better everyone p...
4,741,I wish I was someone else.,I wish I was prettier. I wish I didn’t feel li...,Throwawaybluestahli,1,0,https://www.reddit.com/r/depression/comments/f...,wish wa prettier wish feel like burden wish br...,wish wa someone else,throwaway bluest ahli,321,26,throwaway bluest ahli wish wa prettier wish fe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,1130,My dad is offering us ice cream now,Every night it’s “you guys want ice cream?” Af...,kayoreddit,5,1,https://www.reddit.com/r/SuicideWatch/comments...,every night guy want ice cream dinner everyone...,dad offering u ice cream,kay reddit,585,35,kay reddit every night guy want ice cream dinn...
1512,1294,My mother confessed to me that she attempt to ...,"I would like to say that I was shook, but I kn...",wicho_96,3,1,https://www.reddit.com/r/SuicideWatch/comments...,would like say wa shook knew ha habit consumin...,mother confessed attempt commit suicide,wi cho 96,255,60,wi cho 96 would like say wa shook knew ha habi...
1513,860,"21 years old, make minimum wage, few friends, ...",I can't take this anymore. I've been wanting t...,PoorlyTimed_Meme,0,0,https://www.reddit.com/r/depression/comments/f...,take anymore wanting buy pocket pistol similar...,21 year old make minimum wage friend take one ...,poorly timed meme,662,89,poorly timed meme take anymore wanting buy poc...
1514,1459,How do I kill myself?,emptypost,wpmatt44,10,1,https://www.reddit.com/r/SuicideWatch/comments...,emptypost,kill,wp matt 44,9,21,wp matt 44 emptypost kill


In [86]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\anca.ilicea/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/r

In [88]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
# inputs = tokenizer(df_train, padding="max_length", truncation=True)







100%|██████████| 2/2 [00:00<00:00,  2.72ba/s]




100%|██████████| 1/1 [00:00<00:00,  7.29ba/s]


In [89]:
# split dataset
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']
print(train_dataset)

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1516
})


In [90]:
# set TrainingArguments
training_args = TrainingArguments(
    'train_epoch',
    evaluation_strategy='epoch',
    num_train_epochs=2
)

# load metric
metric = datasets.load_metric('accuracy')
# configuration = model.config

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [91]:
# set train params

from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments("test_trainer")

trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset, 
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# TRAIN
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 1516
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 380












KeyboardInterrupt: 